# Build a Chatbot

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


### Quickstart

In [ ]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

··········


In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Manjit")])

AIMessage(content="Hello Manjit! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 16, 'total_tokens': 43, 'completion_time': 0.021332973, 'prompt_time': 0.003733692, 'queue_time': None, 'total_time': 0.025066665000000002}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-1b7aaa80-c263-425f-ac0f-f56e9148d48d-0', usage_metadata={'input_tokens': 16, 'output_tokens': 27, 'total_tokens': 43})

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I apologize, but I'm a large language model, I don't have the ability to know your personal information, including your name. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to know it!", response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 15, 'total_tokens': 85, 'completion_time': 0.055523356, 'prompt_time': 0.003525278, 'queue_time': None, 'total_time': 0.059048634}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-59a6560f-1927-40ea-b507-71ee02b92ddb-0', usage_metadata={'input_tokens': 15, 'output_tokens': 70, 'total_tokens': 85})

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Manjit"),
        AIMessage(content="Hello Manjit! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content="I'm glad you asked! Your name is Manjit!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 42, 'total_tokens': 55, 'completion_time': 0.009512626, 'prompt_time': 0.011603697, 'queue_time': None, 'total_time': 0.021116323}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-b2a1d9a4-8951-4c90-9773-5fe527875927-0', usage_metadata={'input_tokens': 42, 'output_tokens': 13, 'total_tokens': 55})

### Message History

In [ ]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "ABC2"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Manjit")],
    config=config,
)

response.content

'Hi Manjit! Nice to meet you! Is there something I can help you with or would you like to chat?'

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I remember! Your name is Manjit!'

In [ ]:
config = {"configurable": {"session_id": "ABC3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm happy to help! However, I'm a large language model, I don't have the ability to know your name or any personal information about you. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. Can you please tell me your name?"

In [ ]:
config = {"configurable": {"session_id": "ABC2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Manjit!'

### Prompt templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm manjit")]})

response.content

"Nice to meet you, Manjit! It's great to have you here. How can I help you today? Do you have a particular question or topic you'd like to discuss? I'm all ears (or rather, all text) and ready to assist you!"

In [ ]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "ABC4"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Raj")],
    config=config,
)

response.content

"Hello Raj! It's nice to meet you! I'm here to help you with any questions or topics you'd like to discuss. What's on your mind today?"

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Raj!'

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm manjit")], "language": "Hindi"}
)

response.content

"नमस्ते मनजीत! (Namaste Manjit!) मैं आपकी सहायता के लिए यहाँ हूँ! क्या आपके लिए कुछ मदद करना है या कोई सवाल है? (I'm here to help! Is there anything I can help you with or do you have a question?)"

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [ ]:
config = {"configurable": {"session_id": "ABC5"}}

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Sinoy")], "language": "Assamese"},
    config=config,
)

response.content

'হ্যালো সিনয়! আমি তোমার সহায় হিসেবে থাকি। কোনো প্রশ্ন আছে বা আসেন?'

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Assamese"},
    config=config,
)

response.content

'তোমার নাম সিনয়!'

### Managing Conversation History

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm manjit"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm manjit"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

'I remember! Your name is Manjit!'

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me "whats 2 + 2"'

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "ABC7"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'I remember! Your name is Manjit!'

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked me what 2 + 2 is, and I told you the answer is 4!'

### Streaming

In [ ]:
config = {"configurable": {"session_id": "abc8"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm manjit. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Man|jit|!| Here|'s| one|:

|Why| did| the| scare|crow| win| an| award|?

|Because| he| was| outstanding| in| his| field|!| (|get| it|?)

|Hope| that| made| you| smile|!||